In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from queue import PriorityQueue


R, L, U, D = 1.1, 0.1, 1.1, 0.1 #R if i < i + 1, U if j < j + 1
N = 20
T = 15 #time

In [ ]:
def findMin(N, waitingTimesH, waitingTimesV):
  minWT = waitingTimesH[0][0]
  posX, posY = 0, 0
  isHorizantalEdge = True

  for i in range(2*N):
    for j in range(2*N + 1):
      if (waitingTimesH[i][j] < minWT):
        minWT = waitingTimesH[i][j]
        posX = i
        posY = j
        isHorizantalEdge = True
  for i in range(2 * N + 1):
    for j in range(2 * N):
      if (waitingTimesV[i][j] < minWT):
        minWT = waitingTimesV[i][j]
        posX = i
        posY = j
        isHorizantalEdge = False 

  return minWT, posX, posY, isHorizantalEdge

In [ ]:
def printArr(arr):
  for i in range(len(arr[0])):
    line = ""
    for j in range(len(arr)):
      line += str(arr[j][i]) + " "
    print(line)
  print("\n\n\n")

In [ ]:
def simulate():
  grid = [ [0, 0]*(2 * N + 1) for i in range(2 * N + 1)]
  waitingTimesH, waitingTimesV = [ [0]*(2 * N + 1) for i in range(2 * N)], [ [0]*(2 * N) for i in range(2 * N + 1)] 
  #waitingTimesH[i][j] is edge from (i, j) to (i + 1, j)

  for i in range (2 * N + 1):
    for j in range (2 * N + 1):
      grid[i][j] = [i, j]


  posNN = [[N, N]]
  times = [0]
  timeElapsed = 0;
  count = 0


  #intial alarm clocks
  for i in range(2*N):
    for j in range(2*N + 1):
      waitingTimesH[i][j] = -1/R * np.log(np.random.uniform(0, 1))
  for i in range(2 * N + 1):
    for j in range(2 * N):
      waitingTimesV[i][j] = -1/U * np.log(np.random.uniform(0, 1))

  while (timeElapsed < T):
    count += 1
    minWT, posX, posY, isHorizantalEdge = findMin(N, waitingTimesH, waitingTimesV)

    # print("count == " + str(count) + ", timeElapsed == " + str(timeElapsed))
    # print("horizontal waiting times: ")
    # printArr(waitingTimesH)
    # print("vertical waiting times: ")
    # printArr(waitingTimesV)
    # print(posX, posY)

    timeElapsed = minWT
    #print(timeElapsed)


    #record change
    if (grid[posX][posY] == [N, N] and isHorizantalEdge):
      posNN.append([posNN[-1][0] + 1, posNN[-1][1]])
      times.append(timeElapsed)
    elif (grid[posX][posY] == [N, N] and (not isHorizantalEdge)):
      posNN.append([posNN[-1][0], posNN[-1][1] + 1])
      times.append(timeElapsed)
    elif (posX < 2 * N and grid[posX + 1][posY] == [N, N] and (isHorizantalEdge)):
      posNN.append([posNN[-1][0] - 1, posNN[-1][1]])
      times.append(timeElapsed)
    elif (posY < 2 * N and grid[posX][posY + 1] == [N, N] and (not isHorizantalEdge)):
      posNN.append([posNN[-1][0], posNN[-1][1] - 1])
      times.append(timeElapsed)


    #switch on the number line
    if (posX < 2*N and isHorizantalEdge):
      temp = grid[posX][posY]
      grid[posX][posY] = grid[posX + 1][posY]
      grid[posX + 1][posY] = temp
    if (posY < 2*N and (not isHorizantalEdge)):
      temp = grid[posX][posY]
      grid[posX][posY] = grid[posX][posY + 1]
      grid[posX][posY + 1] = temp

    #fixing waiting times


    if (isHorizantalEdge):
      #horizantal fixes
      if (posX > 0):
        waitingTimesH[posX - 1][posY] = waitingTimesH[posX][posY]
        waitingTimesH[posX - 1][posY] += -1/R * np.log(np.random.uniform(0, 1)) if grid[posX-1][posY][0] < grid[posX][posY][0] else -1/L * np.log(np.random.uniform(0, 1))
      if (posX + 1 < 2*N):
        waitingTimesH[posX + 1][posY] = waitingTimesH[posX][posY]
        waitingTimesH[posX + 1][posY] += -1/R * np.log(np.random.uniform(0, 1)) if grid[posX + 1][posY][0] < grid[posX + 2][posY][0] else -1/L * np.log(np.random.uniform(0, 1))
      #vertical fixes
      if (posY > 0):
        waitingTimesV[posX][posY - 1] = waitingTimesH[posX][posY]
        waitingTimesV[posX][posY - 1] += -1/U * np.log(np.random.uniform(0, 1)) if grid[posX][posY - 1][1] < grid[posX][posY][1] else -1/D * np.log(np.random.uniform(0, 1))
        if (posX + 1 < 2 * N):
          waitingTimesV[posX + 1][posY - 1] = waitingTimesH[posX][posY]
          waitingTimesV[posX + 1][posY - 1] += -1/U * np.log(np.random.uniform(0, 1)) if grid[posX + 1][posY - 1][1] < grid[posX + 1][posY][1] else -1/D * np.log(np.random.uniform(0, 1))
      if (posY < 2 * N):
        waitingTimesV[posX][posY] = waitingTimesH[posX][posY]
        waitingTimesV[posX][posY] += -1/U * np.log(np.random.uniform(0, 1)) if grid[posX][posY][1] < grid[posX][posY + 1][1] else -1/D * np.log(np.random.uniform(0, 1))
        if (posX + 1 < 2 * N):
          waitingTimesV[posX + 1][posY] = waitingTimesH[posX][posY]
          waitingTimesV[posX + 1][posY] += -1/U * np.log(np.random.uniform(0, 1)) if grid[posX + 1][posY][1] < grid[posX + 1][posY + 1][1] else -1/D * np.log(np.random.uniform(0, 1))
      waitingTimesH[posX][posY] += -1/R * np.log(np.random.uniform(0, 1)) if grid[posX][posY][1] < grid[posX + 1][posY][1] else -1/L * np.log(np.random.uniform(0, 1))


    else:
      #vertical fixes
      if (posY > 0):
        waitingTimesV[posX][posY - 1] = waitingTimesV[posX][posY]
        waitingTimesV[posX][posY - 1] += -1/U * np.log(np.random.uniform(0, 1)) if grid[posX][posY - 1][1] < grid[posX][posY][1] else -1/D * np.log(np.random.uniform(0, 1))
      if (posY + 1 < 2*N):
        waitingTimesV[posX][posY + 1] = waitingTimesV[posX][posY]
        waitingTimesV[posX][posY + 1] += -1/U * np.log(np.random.uniform(0, 1)) if grid[posX][posY + 1][1] < grid[posX][posY + 2][1] else -1/D * np.log(np.random.uniform(0, 1))
      #horizantol fixes
      if (posX > 0):
        waitingTimesH[posX - 1][posY] = waitingTimesV[posX][posY]
        waitingTimesH[posX - 1][posY] += -1/R * np.log(np.random.uniform(0, 1)) if grid[posX - 1][posY][0] < grid[posX][posY][0] else -1/L * np.log(np.random.uniform(0, 1))
        if (posY + 1 < 2 * N):
          waitingTimesH[posX - 1][posY + 1] = waitingTimesV[posX][posY]
          waitingTimesH[posX - 1][posY + 1] += -1/R * np.log(np.random.uniform(0, 1)) if grid[posX - 1][posY + 1][0] < grid[posX - 1][posY + 1][0] else -1/L * np.log(np.random.uniform(0, 1))
      if (posX  < 2 * N):
        waitingTimesH[posX][posY] = waitingTimesV[posX][posY]
        waitingTimesH[posX][posY] += -1/R * np.log(np.random.uniform(0, 1)) if grid[posX][posY][0] < grid[posX + 1][posY][0] else -1/L * np.log(np.random.uniform(0, 1))
        if (posY + 1 < 2 * N):
          waitingTimesH[posX][posY + 1] = waitingTimesV[posX][posY]
          waitingTimesH[posX][posY + 1] += -1/R * np.log(np.random.uniform(0, 1)) if grid[posX][posY + 1][0] < grid[posX + 1][posY + 1][0] else -1/L * np.log(np.random.uniform(0, 1))
      waitingTimesV[posX][posY] += -1/U * np.log(np.random.uniform(0, 1)) if grid[posX][posY][0] < grid[posX][posY + 1][0] else -1/D * np.log(np.random.uniform(0, 1))



   

  print(count)
  return times, posNN
  #plt.plot(times, posN)
  #plt.show()



In [ ]:
numPaths = 1

plt.figure(figsize = (16, 16))

for i in range (numPaths):
  print(i)
  times, posNN = simulate()
  posNN = np.array(posNN)
  plt.plot(posNN[:, 0], posNN[:, 1])

plt.show()